# Environment setup

Sets Together AI API key for this session.

In [ ]:
import os

TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

if not TOGETHER_API_KEY:
    raise ValueError("Set TOGETHER_API_KEY in your environment before running this notebook.")

os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY
os.environ["OPENAI_API_KEY"] = TOGETHER_API_KEY  # reused by LangChain ChatOpenAI
print("Together API key set for this session.")


In [ ]:
import os
import builtins

TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
if not TOGETHER_API_KEY:
    raise ValueError("Set TOGETHER_API_KEY in your environment before running this notebook.")

# Make suspicious_agent importable (it expects `openai_key` defined at import time)
builtins.openai_key = TOGETHER_API_KEY

from suspicious_agent import BankingTransactionQA

# Monkey-patch to support Together provider

def _patched_initialize_llm(self, provider, model, api_key, temperature):
    provider = provider.lower()
    if provider == "openai":
        from langchain_openai import ChatOpenAI
        if api_key:
            os.environ["OPENAI_API_KEY"] = api_key
        return ChatOpenAI(model=model or "gpt-4o", temperature=temperature)
    if provider == "together":
        try:
            from langchain_together import ChatTogether
        except ImportError as exc:
            raise ImportError("Install langchain-together to use the Together provider.") from exc
        if api_key:
            os.environ["TOGETHER_API_KEY"] = api_key
        return ChatTogether(model=model or "openai/gpt-oss-120b", temperature=temperature)
    raise ValueError(f"Unsupported LLM provider: {provider}")

BankingTransactionQA._initialize_llm = _patched_initialize_llm

qa = BankingTransactionQA(
    uri="bolt://7.tcp.eu.ngrok.io:10126",
    username="neo4j",
    password="12345678",
    database="transactionsnew",
    llm_provider="together",
    model="openai/gpt-oss-120b",
    api_key=TOGETHER_API_KEY,
    temperature=0,
)

print("BankingTransactionQA initialized with Together.")


In [ ]:
from together import Together

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))
response = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[
        {
            "role": "user",
            "content": "What are some fun things to do in New York?"
        }
    ]
)
print(response.choices[0].message.content)
